In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords


from gensim.models import KeyedVectors
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import gensim.downloader as api
import multiprocessing


import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import time
from tqdm import tqdm

In [2]:
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/hevelyncarvalho/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /home/hevelyncarvalho/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/hevelyncarvalho/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
data = np.array([ 
      'Comecei um novo projeto em Rails e estou adorando!!',
      'Nossa, muito chato trabalhar com Rails! Prefiro codar em Python... Muito mais facil.',
      'Estou querendo migrar para o time de Data Engineering, quero começar a trabalhar com Airflow',
      'Airflow é uma tecnologia muito inovadora, automatiza diversos processos'
])

In [4]:

def preprocessing(text):
    text = text.lower()
    tokens = word_tokenize(text)
    stopWords = set(stopwords.words('portuguese'))
    words = [WordNetLemmatizer().lemmatize(w) for w in tokens if (w.isalpha() and w not in stopWords)]
    return words

l = list((map(preprocessing, data))) # pre-processed dataset
idx = np.array([index for index,a in enumerate(l)])  #  if len(a) > 5
docs_pp_tokens = np.array(l)[idx]  # sem documentos vazios

In [5]:
taggedDocs = [TaggedDocument(doc, [i]) for i, doc in enumerate(docs_pp_tokens)] #matriz de documentos com as tags, cada linha é um vetor dos tokens de cada doc
taggedDocs    

[TaggedDocument(words=['comecei', 'novo', 'projeto', 'rail', 'adorando'], tags=[0]),
 TaggedDocument(words=['chato', 'trabalhar', 'rail', 'prefiro', 'codar', 'python', 'facil'], tags=[1]),
 TaggedDocument(words=['querendo', 'migrar', 'time', 'data', 'engineering', 'quero', 'começar', 'trabalhar', 'airflow'], tags=[2]),
 TaggedDocument(words=['airflow', 'tecnologia', 'inovadora', 'automatiza', 'diversos', 'processos'], tags=[3])]

In [6]:


d2v = Doc2Vec(dm=0, vector_size=50, window=2, min_count=1, sample=6e-5, alpha=0.03, min_alpha=0.0007)
d2v.build_vocab(documents=taggedDocs)
total = d2v.corpus_count
#vocab = [TaggedDocument(w, [i]) for i, w in enumerate(nilc.vocab.keys())]
#d2v.build_vocab(vocab, update=True)

d2v.train(taggedDocs, total_examples=total, epochs=100)
docs = np.array([d2v.docvecs[i] for i in range(len(taggedDocs))])

In [ ]:
text = "Estou usando muito Airflow ultimamente"
tokens = preprocessing(text)
new_vector = d2v.infer_vector(tokens)

In [ ]:
sims = np.array(d2v.docvecs.most_similar([new_vector]))
idx = sims[:,0].astype(int)
data[idx]